---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [38]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])



### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [45]:
def answer_one():
    B = nx.Graph()
    G2_df = pd.read_csv('Employee_Movie_Choices.txt',header = None,delimiter="\t",names = ['Employee','Movie'],skiprows = 1  )
    B.add_nodes_from(G2_df['Employee'], bipartite = 0)
    B.add_nodes_from(G2_df['Movie'], bipartite = 1)
    
    edges = list(zip(G2_df['Employee'],G2_df['Movie']))
    B.add_edges_from(edges)

    #G2 = nx.read_adjlist('Employee_Movie_Choices.txt',delimiter = "\t",create_using= nx.DiGraph())
    return B
answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [46]:
def answer_two():
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.add_node(node,type='employee')
        else:
            G.add_node(node, type = 'movie')
    return G
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [49]:
def answer_three():
    G = answer_two()
    
    return bipartite.weighted_projected_graph(G,employees)
answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [88]:
def answer_four2():
        
    df = pd.read_csv('Employee_Relationships.txt',header = None ,delimiter = '\t', names=['Emp1', 'Emp2','Rel'])
    
    c_df = pd.DataFrame(answer_three().edges(data=True))
    c_df.columns = ['Emp1','Emp2','cm']
    c_df['cm'] = c_df['cm'].map(lambda x: x['weight'])
    
    #df = pd.merge(df,c_df, how= 'outer', left_on =['Emp1','Emp2'],right_on =['Emp1','Emp2'])
    merge_c_df = pd.concat([c_df, c_df.rename(columns = {'Emp1':'Emp2', 'Emp2':'Emp1'})])

    df = (df.merge(merge_c_df, left_on = ['Emp1', 'Emp2'],
                        right_on = ['Emp1', 'Emp2']))
    #.drop(['Emp1', 'Emp2'], axis=1))
    
    #df['cm'].replace(np.nan, 0, inplace= True)
    #df = df[ ['Rel','cm']]
    return df#float(df.corr('pearson')['Rel']['cm'])
answer_four2()

0.9060934635351607

In [97]:
def answer_four():
    G = answer_three()
    df = pd.read_csv('Employee_Relationships.txt',header = None ,delimiter = '\t', names=['Emp1', 'Emp2','Rel'])
    df['Common Movies'] =  df.apply(lambda x: G[ x['Emp1'] ][ x['Emp2'] ]['weight'] if G.has_edge(x['Emp1'],x['Emp2']) else 0 ,axis = 1)
    df = df[ ['Rel','Common Movies']]
    return float(df.corr('pearson')['Rel']['Common Movies'])
answer_four()

0.7883962221733476